<a href="https://colab.research.google.com/github/daniyal9538/GeneralProjects/blob/master/cleanMineData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import csv

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving anonReviews (1).json to anonReviews (1).json
User uploaded file "anonReviews (1).json" with length 11623976 bytes


In [0]:
def upload():
  with open(reviewFile, encoding="utf8") as f:
    data = json.load(f)
  return data

In [0]:
def dump(newData):
  with open(newRevFile, 'w') as f:
    json.dump(newData, f, indent=4)

In [0]:
def removePhotos(data):
  count = 0
  for reviews in data:
    try:
      del reviews['photos']
    except KeyError:
      pass
    count +=1
  print(count)
  return data

In [0]:
def cleanData(data):
  count = 0
  cat = 'category'
  name = 'name'
  price = 'price'
  taste = 'taste'
  value = 'valueForMoney'
  for reviews in data:
    try:
      temp = reviews['likedBy']
    except KeyError:
      reviews.update({'likedBy': []})
    for dish in reviews['dishes']:
      try:
        temp = dish[cat]
      except KeyError:
        dish.update({cat:None})
        count+=1
      
        
      try:
        temp = dish[name]
      except KeyError:
        dish.update({name:None})
        count+=1
        
      try:
        temp = dish[price]
      except KeyError:
        dish.update({price:None})
        count+=1
        
      try:
        temp = dish[taste]
      except KeyError:
        dish.update({taste:None})
        count+=1
        
      try:
        temp = dish[value]
      except KeyError:
        dish.update({value:None})
        count+=1
  print(count)
  return data

In [0]:
def mineData(data):
  for reviews in data:
 
    tempPrice = 0
    tempTaste = 0
    aggValue = 0
    aggRestScore = 0
    flag = False
    likeCount = len(reviews['likedBy'])
    dishCount = len(reviews['dishes'])
    for dish in reviews['dishes']:
      try:
        tempPrice+=dish['price']
      except TypeError:
        flag = True
      tempTaste += dish['taste']
      if (dish['valueForMoney']=='yes'):
        aggValue +=1
      else:
        aggValue -= 1
    if (flag==False):
      avgPrice = float(tempPrice/dishCount)
    else:
      avgPrice = None
    avgTaste = float(tempTaste/dishCount)
    for i in range(3):
      aggRestScore+=list(reviews['restaurantRatingValues'][i].values())[1]
    aggRestScore = float(aggRestScore/3)
    reviews.update({'likeCount': likeCount})
    reviews.update({'dishCount': dishCount})
    reviews.update({'avg price rating': avgPrice})
    reviews.update({'avg taste rating': avgTaste})
    reviews.update({'avg value rating': aggValue})
    reviews.update({'agg restaurant rating': aggRestScore})
  return data

In [0]:
def writeToCSV(data, name):
  fields = ['revID', 'restID', 'uID', 'score', 'time', 'dishCount', 'likeCount', 'avgDishTaste', 'avgDishPrice', 'restValueForMoney']
  fields.append('restService')
  fields.append('restAmbiance')
  fields.append('restSB')
  fields.append('aggRestRating')

  
  with open(name, 'a') as writeFile:
    writer = csv.writer(writeFile, lineterminator='\n')
    writer.writerow(fields)
    for review in data:
      t = []
      t.append(review['_key'])
      t.append(review['restaurant'])
      t.append(review['user'])
      t.append(review['score'])
      t.append(review['time'])
      t.append(review['dishCount'])
      t.append(review['likeCount'])
      t.append(review['avg taste rating'])
      t.append(review['avg price rating'])
      t.append(review['avg value rating'])
      t.append(review['restaurantRatingValues'][0]['value'])
      t.append(review['restaurantRatingValues'][1]['value'])
      t.append(review['restaurantRatingValues'][2]['value'])
      t.append(review['agg restaurant rating'])
      
      writer.writerow(t)

In [0]:
reviewFile = 'reviews2.json'
restaurantFile = 'restaurantsProduction.json'
newRevFile='reviews2.json'

In [0]:
data = upload()

In [0]:
data = removePhotos(data)

In [0]:
data = cleanData(data)

In [0]:
data = mineData(data)

In [0]:
dump(data)

In [0]:
writeToCSV(data, 'data.csv')